# Padrão de projeto MVC

O conteúdo deste notebook provem principalmente de:
 - https://pt.wikipedia.org/wiki/MVC
 - https://realpython.com/the-model-view-controller-mvc-paradigm-summarized-with-legos/
 - https://www.giacomodebidda.com/mvc-pattern-in-python-introduction-and-basicmodel/


Um padrão de projeto (*design pattern*) é uma solução geral para um problema que ocorre com frequência. 


O modelo MVC (**Model, View, Controller**) separa partes distintas do projeto reduzindo suas dependências ao máximo: o front-end responsável pela interação com o usuário e o back-end responsável pelos dados e a lógica do sistema. 


As três camadas do modelo são:
 - **Model** (Lógica da aplicação): Consiste nos dados da aplicação, regras de negócios, lógica e funções. Permite o acesso para os dados serem coletados, gravados e exibidos. Esta componente **modela o problema que está se tentando resolver**. 
 - **View** (Apresentação): Saída e representação de dados, por exemplo, uma GUI (Interface gráfica de usuário), página web, impressão na tela, gráfico de barras, etc. Um modelo pode ter várias visões. 
 - **Controller** (Controlador): mediação da entrada, convertendo-a em comandos para o modelo ou visão. Neste componente são manipulados os dados que o usuário insere e atualiza o Modelo de acordo.

Vantagens:
 - Vários views para o mesmo modelo
 - Facilita o reuso do código
 - Partes da aplicação podem ser alteradas sem a necessidade de alterar outras.
 - Baixo **acoplamento** (acomplamento é o grau em que uma classe conhece a outra). As interface entre as classes está melhor definida. 
 - Alta **coesão** (ou seja, as classes têm propósitos bem definidos). 


## Exemplo

Considere o exemplo no arquivo pessoa.zip 

 - pessoa.py: Classe Pessoa (Modelo). 
 - pessoaCtr.py: Classe PessoaCtr ( Controlador)
 - pessoaView.py: Classe PessoaView (View)

Algumas coisas importantes:
 - A classe do modelo define os atributos a serem armazenados e mantem um "banco de dados" das pessoas (neste exemplo simples, o banco de dados é um simples dicionário)
 - O modelo oferece métodos para manipular os objetos (Pessoas)
 - A classe *View* deve lidar com a visualização dos dados e as respostas do sistema (por exemplo, listas de pessoas, mensagens de erro, etc). 
 - Note que a classe View não acessa direitamente o banco de dados
 - A classe do Controlador lida com as ações do usuário. Note que essa classe utiliza a classe do Modelo para recuperar os dados e utiliza a classe View para visualizar informações. 


> What you need to ask yourself to define if your app is MVC is the following:

>  - If I change something in the view do I break anything in the model?
>  - If I change something in the model do I break anything in the view?
>  - Is the controller communicating everything in both view and model so that they don't have to communicate with each other?

> If nothing breaks and the controller does all of the communication then yes, your application is MVC.
> https://stackoverflow.com/questions/38042632/mvc-the-simplest-example

No nosso exemplo:
 - Podemos mudar qualquer uma das classes e o sistema continua funcionando. Por exemplo, podemos trocar o modelo (adicionando uma conexão a um banco de dados) e as outras duas classes continuam funcionando. Podemos trocar o View por uma interface gráfica (o página HTML) e as outras duas continuam funcionando. 
 - A classe View não precisa saber como foi implementado o modelo (só precisa saber os atributos/properties dessa classe)
 - A classe do Modelo não precisa saber como os objetos vão ser visualizados. 

# JSON e a biblioteca TinyBD

JSON (JavaScript Object Notation) é um formato para trocar dados entre sistemas. O formato é muito simples, por exemplo:

```
{
 "cod": 1, 
 "nome": "produto1", 
 "preco": 250
}
```

muito parecido a...... um dicionário!
```
d = {"cod" : 1, "nome" : "produto1", "preco" : 250}
```


Vamos utilizar a biblioteca *json* e  *TinyDB* (https://pypi.org/project/tinydb/) para manipular e armazenar arquivos JSON.

## json.dumps
A função ```json.dumps``` converte tipos como ```int```, ```float```, ```str```, ```list``` e ```dict``` em strings no formato JSON. 

In [ ]:
import json

print(json.dumps(3)) 
print(json.dumps(3.14)) 
print(json.dumps((1,"string"))) # Tuplas são representadas como listas em JSON
print(json.dumps('alo')) 
print(json.dumps([1,2,3])) 
print(json.dumps({'a': 1, 'b':2, 'c':[1,2,3]})) 

#Exemplo com estruturas aninhadas 
print(json.dumps({'a': 1, 'b':2, 'c': {'x': 'alo', 'y':[1,2,3]}})) 


## json.dumps com classes


In [ ]:
class A:
    def __init__(self):
        self.x = 3
        self.y = 'alo'

va = A()
json.dumps(va) #Erro! Objetos tipo A não podem ser convertidos

In [ ]:
# Em Python podemos retornar os atributos de  um objeto
print(va.__dict__)
# Uma forma mais pythonica
print(vars(va))

In [ ]:
class Produto:
    def __init__(self, cod, nome, preco,cores):
        self._cod = cod
        self._nome = nome
        self._preco = preco
        self._cores = cores

    def toDict(self):
        '''Retorna um dicionario com os atributos'''
        return vars(self)

P = Produto(1,'produto1',400, ['branco', 'verde'])

# Um produto no formato JSON
print(json.dumps(P.toDict()))


Mas será que precisamos implementar ```toDict``` para todas as classes do sistemas ? É muito mais simples estender a classe ```json.JSONEncoder```

In [ ]:
class MyEncoder(json.JSONEncoder):
     def default(self, o):
            '''Método que define como "serializar" um objeto'''
            return vars(o)

class Cor:
    def __init__(self, nome):
        self._nome = nome

class Produto:
    def __init__(self, cod, nome, preco,cores):
        self._cod = cod
        self._nome = nome
        self._preco = preco
        self._cores = cores

Branco = Cor('branco')
Verde = Cor('verde')
Vermelho = Cor('vermelho')

P1 = Produto(1,'produto1',400, [Branco])
P2 = Produto(1,'produto2',200, [Verde, Vermelho])

# Serializar uma Cor
print(json.dumps(Branco, cls=MyEncoder))

print(json.dumps(P1, cls=MyEncoder))
print(json.dumps(P2, cls=MyEncoder))
# Serializar uma lista de produtos
print(json.dumps([P1, P2], cls=MyEncoder))

# json.load

Dada  uma string utilizando o formato JSON podemos construir um dicionário em Python

In [ ]:
s = '{"_cod": 1, "_nome": "produto1", "_preco": 400, "_cores": [{"_nome": "branco"}]}'

# Dicionario com os atributos
dados = json.loads(s)

print(dados)
print(dados["_cod"])
print(dados["_cores"])

No código a seguir:
 - ```MyEncoder``` implementa o método ```default``` para converter um objeto em um dicionário. Note que, além dos atributos do objeto, um novo atributo é adicionado: ```tipo```. 
 - O método ```decode``` recebe como parâmetro um dicionário (que contem o atributo ```tipo```) e constrói um objeto da classe apropriada. 

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, o):
        d = vars(o) # Atributos do objeto
        d['tipo'] = o.__class__.__name__  #Adicionar o tipo de objeto
        return d

    @staticmethod
    def decode(d):
        '''recebe como parâmetro um dicionario'''
        if d['tipo'] == 'Cor':
            return Cor(d['_nome'])
        if d['tipo'] == 'Produto':
            return Produto(d['_cod'], d['_nome'], d['_preco'],d['_cores'])

class Cor:
    def __init__(self, nome):
        self._nome = nome


    def __repr__(self):
        return f'Cor({self._nome})'

class Produto:
    def __init__(self, cod, nome, preco,cores):
        self._cod = cod
        self._nome = nome
        self._preco = preco
        self._cores = cores

    def __repr__(self):
        return f'Produto{self._cod, self._nome, self._preco, self._cores}'



C1 = Cor('verde')
C2 = Cor('branco')
C3 = Cor('preto')

P = Produto(1, 'produto1', 200, [C1 , C3])

s = json.dumps(P, cls=MyEncoder)
# Note que s contem o atributo tipo
print(s)
#Criar um objeto Pessoa a partir da string JSON
P2 = json.loads(s, object_hook=MyEncoder.decode)
print(P2)

## Biblioteca TinyDB
A biblioteca ```TinyDB``` permite armazenar, adicionar, atualizar e remover objetos no formato JSON.


In [ ]:
from tinydb import TinyDB, Query
import json

print('--- inserir elementos ---')
with  TinyDB('dados-exemplo.json') as db:
    # Inserir um objeto no arquivo dados-exemplo.json
    # Insert recebe como parâmetro um dicionário
    db.insert({'cod':1, 'nome': 'Nome 1'}) 
    db.insert({'cod':2, 'nome': 'Nome 2'}) 
    db.insert({'cod':3, 'nome': 'Nome 3'}) 
    db.insert({'cod':4, 'nome': 'Nome 4'}) 
    db.insert({'cod':4, 'nome': 'Nome 5'}) 
    

print ('--- Consultar o arquivo ---')
with  TinyDB('dados-exemplo.json') as db:
    for d in db:
        print(d) # d é um dicionário

print ('--- Elementos com nome = Nome 1---')
with  TinyDB('dados-exemplo.json') as db:
    Q = Query()
    l = db.search(Q.nome == 'Nome 4')
    for x in l:
        print(x)
        
print ('--- Elementos com cod >= 3 ---')
with  TinyDB('dados-exemplo.json') as db:
    Q = Query()
    l = db.search(Q.cod >= 3)
    for x in l:
        print(x)
        
print ('--- Atualizar o nome  ---')
with  TinyDB('dados-exemplo.json') as db:
    Q = Query()
    # Atualizar nome para os elementos com cod = 3
    l = db.update({'nome' : 'Novo Nome'}, Q.cod == 3)

print ('--- Remover elementos  ---')    
with  TinyDB('dados-exemplo.json') as db:
    Q = Query()
    # Remover elementos
    l = db.remove(Q.cod == 1)
    
    
print ('--- Consultar o arquivo ---')
with  TinyDB('dados-exemplo.json') as db:
    for d in db:
        print(d) # d é um dicionário



## Exemplo
O exemplo a seguir  implementa os métodos ```toDict``` e ```fromDict``` na classe ```Produto``` para converter objetos do tipo ```Produto``` em dicionários e vice-versa utilizando a biblioteca ```json``` como visto na seção anterior. Com esses métodos, podemos utilizar TinyDB para armazenar produtos.

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, o):
        d = vars(o) # Atributos do objeto
        d['tipo'] = o.__class__.__name__  #Adicionar o tipo de objeto
        return d

    @staticmethod
    def decode(d):
        '''recebe como parâmetro um dicionario'''
        if d['tipo'] == 'Cor':
            return Cor(d['_nome'])
        if d['tipo'] == 'Produto':
            return Produto(d['_cod'], d['_nome'], d['_preco'],d['_cores'])

class Cor:
    def __init__(self, nome):
        self._nome = nome

    def __repr__(self):
        return f'Cor({self._nome})'

class Produto:
    def __init__(self, cod, nome, preco,cores):
        self._cod = cod
        self._nome = nome
        self._preco = preco
        self._cores = cores

    def __repr__(self):
        return f'Produto{self._cod, self._nome, self._preco, self._cores}'

    def toDict(self):
        '''Retorna a representacao como dicionario'''
        s = json.dumps(self, cls=MyEncoder)
        return json.loads(s)

    @staticmethod
    def fromDict(d):
        s = json.dumps(d)
        return json.loads(s, object_hook=MyEncoder.decode)


C1 = Cor('verde')
C2 = Cor('branco')
C3 = Cor('preto')

P1 = Produto(1, 'produto1', 200, [C1 , C3])
P2 = Produto(2, 'produto2', 400, [C2 , C3])
P3 = Produto(3, 'produto3', 600, [C2 , C3])
P4 = Produto(4, 'produto4', 800, [C1 , C3])
l = [P1, P2, P3, P4]
# Inserir os elementos no arquivo dados.json
with  TinyDB('produtos.json') as db:
    for x in l:
        db.insert(x.toDict())

# Listar os produtos com preço >= 500
with  TinyDB('produtos.json') as db:
    Q = Query()
    l = db.search(Q._preco >= 500)
    for x in l:
        # convertemos  o dicionário x em um objeto tipo Produto
        p = Produto.fromDict(x)
        print(p)
    


## Exercício

Modifique o exemplo das pessoas para utilizar a biblioteca TinyDB (no Modelo) e armazenar as Pessoas em um arquivo JSON. Note que o sistema deve continuar funcionando sem modificar as classes do controlador e da vista.  